# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import os

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns

In [3]:
from path_constants import H5AD
from nb_utils import describe

from celltype_utils import get_shared_adata

## Read one2one h5ad

In [4]:
adata_shared = get_shared_adata()

/home/olga/miniconda3/envs/immune-evolution/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


# Read parquet files

## File paths

In [5]:
sketch_id = 'alphabet-dayhoff__ksize-51__scaled-10'
sig_outdir_base = "/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures"


parquets = glob.glob(f'/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/*/2--single-cell-kmers/{sketch_id}/hash2kmer__unique_kmers_per_celltype.parquet')
parquets

['/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/1--train-mouse--to-publish/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/2--test-human/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/3--test-bat/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/4--test-lemur/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet',
 '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/1--train-mouse/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet']

In [6]:
# parquets = glob.glob(f'/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/*/2--single-cell-kmers/*/hash2kmer__unique_kmers_per_celltype.parquet')
# parquets

In [ ]:
%%time
dfs = []

for parquet in parquets:
    ! ls -lha $parquet
    %time df = pd.read_parquet(parquet)
    df = df.rename(columns={'broad_group': 'groundtruth_celltype'})
    dfs.append(df)
    
hash2kmer = pd.concat(dfs)
describe(hash2kmer)

-rw-r--r-- 1 olga czb 3.3G Jun 23 11:12 /home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/1--train-mouse--to-publish/2--single-cell-kmers/alphabet-dayhoff__ksize-51__scaled-10/hash2kmer__unique_kmers_per_celltype.parquet


## Read predicted cell types

In [ ]:
predicted_cells_min_5_percent = pd.read_parquet(
    os.path.join(sig_outdir_base, "aggregated-predicted-cells.parquet")
)
predicted_cells_min_5_percent = predicted_cells_min_5_percent.query('sketch_id == @sketch_id')
describe(predicted_cells_min_5_percent)

### Join hash2kmer with predicted cell types

In [ ]:
celltype_predictions = predicted_cells_min_5_percent[
    ["predicted_celltype", "groundtruth_compartment", "predicted_compartment"]
]
celltype_predictions.index = celltype_predictions.index.str.upper()
describe(celltype_predictions)

In [ ]:
hash2kmer.cell_id = hash2kmer.cell_id.str.upper()

In [ ]:
hash2kmer_with_predictions = hash2kmer.join(celltype_predictions, on='cell_id')
describe(hash2kmer_with_predictions)

In [ ]:
hash2kmer_with_predictions['hashval'] = hash2kmer_with_predictions['hashval'].astype(str)

### Write joined to parquet

In [ ]:
hash2kmer_with_predictions.to_parquet(
    os.path.join(sig_outdir_base, f"aggregated-hash2kmer-with-predicted-cells__{sketch_id}.parquet")
)

## Seems like some cell ids aren't there .. why?

In [ ]:
hash2kmer_cell_ids = set(hash2kmer.cell_id)

In [ ]:
adata_shared.obs.index = adata_shared.obs.index.str.upper()

In [ ]:
cell_ids_not_in_hash2kmer = adata_shared.obs.index.difference(hash2kmer_cell_ids)
len(cell_ids_not_in_hash2kmer)

In [ ]:
adata_shared.obs.loc[cell_ids_not_in_hash2kmer].species.value_counts()

In [ ]:
channel_sizes = adata_shared.obs.groupby('channel').size()

In [ ]:
no_id_present_channel_sizes = adata_shared.obs.loc[cell_ids_not_in_hash2kmer].groupby('channel').size()

In [ ]:
no_id_present_channel_sizes

In [ ]:
100 * no_id_present_channel_sizes/channel_sizes

In [ ]:
celltype_hashvals = hash2kmer.groupby(['species', 'groundtruth_celltype']).alignment_status.value_counts()
celltype_hashvals.head()

In [ ]:
celltype_hashvals

In [ ]:
celltype_hashval_percentage = celltype_hashvals.groupby(level=[0, 1]).apply(lambda x: 100 * x/x.sum())
celltype_hashval_percentage.name = 'percentage'
celltype_hashval_percentage = celltype_hashval_percentage.reset_index()
celltype_hashval_percentage

In [ ]:
celltype_hashval_percentage_unaligned = celltype_hashval_percentage.query('alignment_status == "unaligned"')


In [ ]:
order = celltype_hashval_percentage_unaligned.query('species == "bat"').sort_values(
    "percentage"
)["groundtruth_celltype"]
order

In [ ]:
g = sns.catplot(
    data=celltype_hashval_percentage_unaligned,
    y="groundtruth_celltype",
    x="percentage",
    hue="species",
    kind="bar",
    order=order
)
g.set(xscale="log")

In [ ]:
g = sns.catplot(
    data=celltype_hashval_percentage_unaligned,
    y="groundtruth_celltype",
    x="percentage",
    col="species",
    kind="bar",
    order=order
)
g.set(xscale="log")

## Read mouse diagnostic k-mer csvs

In [ ]:
mouse_dir = '/home/olga/data_lg/data_sm_copy/immune-evolution/kmer-signatures/1--train-mouse/'
celltype_kmer_subdir = '5--celltype-kmers--merged-celltype-remove-common-kmers--min-kmer-count--5-percent'

mouse_celltype_kmer_csvs = glob.glob(os.path.join(mouse_dir, celltype_kmer_subdir, sketch_id, 'csvs', '*csv'))
mouse_celltype_kmer_csvs

In [ ]:
%%time

dfs = []

for csv in mouse_celltype_kmer_csvs:
    basename = os.path.basename(csv)
    print('\ncelltype file:', )
    %time df = pd.read_csv(csv)
    df['celltype'] = basename.split('.')[0]
    dfs.append(df)
mouse_celltype_kmers = pd.concat(dfs)
describe(mouse_celltype_kmers)

# Gene orthology

## Assign figure folder

In [ ]:
figure_folder = (
    "/home/olga/googledrive/ImmuneEvolution/figures/kmer_gene_orthology"
)
!mkdir -p $figure_folder

## Use 1:1 orthologs from Anndata of bat-Lemur-human-mouse

In [ ]:
describe(adata_shared.var)

## Assign k-mer type based on alignment status and gene name

In [ ]:
PER_SPECIES_ORTHOLOGOUS_GENES = {
    "bat": set(adata_shared.var["bat__gene_name"]),
    "lemur": set(adata_shared.var["mouse_lemur__gene_name_x-hlm"]),
    "human": set(adata_shared.var.index),
    "mouse": set(adata_shared.var["mouse__gene_name-bat"]),
}
for k, v in PER_SPECIES_ORTHOLOGOUS_GENES.items():
    print(k, len(v))

In [ ]:
%%time

def is_orthologous(query_gene, species, per_species_orthologous_genes=PER_SPECIES_ORTHOLOGOUS_GENES):
    orthologous_genes = per_species_orthologous_genes[species]
    return query_gene in orthologous_genes


def assign_kmer_category(row):
    if row.alignment_status == 'unaligned':
        return 'Not in reference genome'
    elif pd.isnull(row.gene_name):
        return 'In ref genome, not in a gene'
    elif is_orthologous(row.gene_name, row.species):
        return 'In ref genome, in a 1:1 orthologous gene'
    else:
        return 'In ref genome, not in a 1:1 orthologous gene'

hash2kmer['kmer_category'] = hash2kmer.apply(assign_kmer_category, axis=1)
hash2kmer.groupby(['species', 'kmer_category']).size()

In [ ]:
hash2kmer_with_predictions = hash2kmer.join(celltype_predictions, on='cell_id')
describe(hash2kmer_with_predictions)

In [ ]:
kmer_categories = (
    "Not in reference genome",
    "In ref genome, not in a gene",
    "In ref genome, not in a 1:1 orthologous gene",
    "In ref genome, in a 1:1 orthologous gene",
)

In [ ]:
hash2kmer_with_predictions['kmer_category'] = pd.Categorical(hash2kmer_with_predictions['kmer_category'], categories=kmer_categories, ordered=True)

## Write hash2kmer with predictions and orthology to parquet

In [ ]:
hash2kmer_with_predictions['hashval'] = hash2kmer_with_predictions['hashval'].astype(str)

In [ ]:
hash2kmer_with_predictions.to_parquet(
    os.path.join(sig_outdir_base, f"aggregated-hash2kmer-with-predicted-cells--{sketch_id}--with-orthology.parquet")
)

In [ ]:
1+1

## Get number of k-mers in different categories per `groundtruth_celltype`

In [ ]:
%%time

celltype_col = "groundtruth_celltype"

diagnostic_kmers_n_per_category = hash2kmer_with_predictions.groupby(
    ["species", celltype_col, "kmer_category"]
).hashval.nunique()
diagnostic_kmers_n_per_category.name = "percent_kmers"
diagnostic_kmers_n_per_category

In [ ]:
diagnostic_kmers_n_per_category_df = diagnostic_kmers_n_per_category.groupby(level=[0, 1]).apply(lambda x: 100*x/x.sum()).reset_index()
diagnostic_kmers_n_per_category_df

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
diagnostic_kmers_n_per_category_df.query(
    '(species == "lemur") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

# Plot percentage of kmers in ref genome, etc

In [ ]:
figure_folder = '/home/olga/googledrive/kmer-homology-paper/figures/kmer_gene_orthology'
! mkdir $figure_folder

In [ ]:
species_order = 'human', 'lemur', 'mouse', 'bat'
diagnostic_kmers_n_per_category_df['species'] = pd.Categorical(diagnostic_kmers_n_per_category_df['species'], categories=species_order, ordered=True)

In [ ]:
g = sns.catplot(
    data=diagnostic_kmers_n_per_category_df,
    col=celltype_col,
    order=species_order,
    col_wrap=5,
    y="species",
    x="percent_kmers",
    hue="kmer_category",
    hue_order=kmer_categories,
    palette='mako',
    kind='bar',
    height=2,
    linewidth=1,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    title = ax.get_title()
    if title == 'Smooth Muscle and Myofibroblast':
        ax.set_title('Smooth Muscle\nand Myofibroblast', fontsize=10, pad=-20)
pdf = os.path.join(figure_folder, 'unstacked_barplot__col-celltype__y-species__hue-kmer_category.pdf')
g.savefig(pdf)

In [ ]:
g = sns.catplot(
    data=diagnostic_kmers_n_per_category_df,
    hue=celltype_col,
#     col_wrap=2,
    y="species",
    order=species_order,
    x="percent_kmers",
    col="kmer_category",
    col_order=kmer_categories,
    palette='tab10',
    kind='bar',
    height=2.5,
    sharex=True,
        linewidth=.5,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    if ax.is_last_row():
        title = ax.get_title()
        title = title.replace(',', ',\n')
        ax.set_title(title, fontsize=10, pad=-20)
#     if ax.is_first_col():
#         ax.set(xscale='log')
pdf = os.path.join(figure_folder, 'unstacked_barplot__col-kmer_category__y-species__hue-celltype.pdf')
g.savefig(pdf)

In [ ]:
species_order_no_lemur = ['human','mouse', 'bat']

In [ ]:
diagnostic_kmers_n_per_category_df_no_lemur = diagnostic_kmers_n_per_category_df.query(
    "species in @species_order_no_lemur"
)
diagnostic_kmers_n_per_category_df_no_lemur.species = pd.Categorical(
    diagnostic_kmers_n_per_category_df_no_lemur.species,
    categories=species_order_no_lemur,
    ordered=True,
)
describe(diagnostic_kmers_n_per_category_df_no_lemur)

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=2, figsize=(12, 3), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    diagnostic_kmers_n_per_category_df_no_lemur.groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
#         yticks=species_order,
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, 'stacked_barplot__col-celtype__y-species__hue-kmer_category.pdf')
fig.savefig(pdf)

In [ ]:
figure_folder

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=5, figsize=(4.5, 6), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    diagnostic_kmers_n_per_category_df_no_lemur.groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, 'stacked_barplot__col-celtype__y-species__hue-kmer_category__two_column.pdf')
fig.savefig(pdf)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4), sharex=True, sharey=True)


for (species, df), ax in zip(
    diagnostic_kmers_n_per_category_df_no_lemur.groupby("species"), axes.flat
):
    legend = ax.is_last_col()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y=celltype_col,
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=legend,
        ax=ax,
    )
    ax.set(
        ylabel="percentage",
        title=species,
#         yticks=["bat", "human"],
    )
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

sns.despine()
fig.tight_layout()

pdf = os.path.join(figure_folder, 'stacked_barplot__col-species__y-celltype__hue-kmer_category.pdf')
fig.savefig(pdf)

In [ ]:
1+1

## Get number of k-mers in different categories per `predicted_compartment`

In [ ]:
%%time

celltype_col = "predicted_compartment"

compartment_kmers_n_per_category = hash2kmer_with_predictions.groupby(
    ["species", celltype_col, "kmer_category"]
).hashval.nunique()
compartment_kmers_n_per_category.name = "percent_kmers"
compartment_kmers_n_per_category

In [ ]:
compartment_kmers_n_per_category_df = compartment_kmers_n_per_category.groupby(level=[0, 1]).apply(lambda x: 100*x/x.sum()).reset_index()
compartment_kmers_n_per_category_df

In [ ]:
compartment_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
compartment_kmers_n_per_category_df.query(
    '(species == "mouse") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

In [ ]:
compartment_kmers_n_per_category_df.query(
    '(species == "lemur") and (kmer_category == "Not in reference genome")'
).sort_values('percent_kmers')

# Plot percentage of kmers in ref genome, etc

In [ ]:
figure_folder = '/home/olga/googledrive/kmer-homology-paper/figures/kmer_gene_orthology'
! mkdir $figure_folder

In [ ]:
species_order = 'human', 'lemur', 'mouse', 'bat'
compartment_kmers_n_per_category_df['species'] = pd.Categorical(compartment_kmers_n_per_category_df ['species'], categories=species_order, ordered=True)

In [ ]:
g = sns.catplot(
    data=compartment_kmers_n_per_category_df,
    col=celltype_col,
    order=species_order,
    col_wrap=5,
    y="species",
    x="percent_kmers",
    hue="kmer_category",
    hue_order=kmer_categories,
    palette='mako',
    kind='bar',
    height=2,
    linewidth=1,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    title = ax.get_title()
    if title == 'Smooth Muscle and Myofibroblast':
        ax.set_title('Smooth Muscle\nand Myofibroblast', fontsize=10, pad=-20)
pdf = os.path.join(figure_folder, f'unstacked_barplot__col-celltype__y-species__hue-kmer_category__{celltype_col}.pdf')
g.savefig(pdf)

In [ ]:
g = sns.catplot(
    data=compartment_kmers_n_per_category_df,
    hue=celltype_col,
#     col_wrap=2,
    y="species",
    order=species_order,
    x="percent_kmers",
    col="kmer_category",
    col_order=kmer_categories,
    palette='tab10',
    kind='bar',
    height=2.5,
    sharex=True,
        linewidth=.5,
    edgecolor='white'
#     legend=True,
)
g.set_titles('{col_name}')
for ax in g.axes.flat:
    if ax.is_last_row():
        title = ax.get_title()
        title = title.replace(',', ',\n')
        ax.set_title(title, fontsize=10, pad=-20)
#     if ax.is_first_col():
#         ax.set(xscale='log')
pdf = os.path.join(figure_folder, f'unstacked_barplot__col-kmer_category__y-species__hue-celltype__{celltype_col}.pdf')
g.savefig(pdf)

In [ ]:
species_order_no_lemur = ['human','mouse', 'bat']

In [ ]:
compartment_kmers_n_per_category_df_no_lemur = compartment_kmers_n_per_category_df.query(
    "species in @species_order_no_lemur"
)
compartment_kmers_n_per_category_df_no_lemur.species = pd.Categorical(
    compartment_kmers_n_per_category_df_no_lemur.species,
    categories=species_order_no_lemur,
    ordered=True,
)
describe(compartment_kmers_n_per_category_df_no_lemur)

In [ ]:
compartment_kmers_n_per_category_df_no_lemur.query('kmer_category == "Not in reference genome"')

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=2, figsize=(12, 3), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    compartment_kmers_n_per_category_df_no_lemur.groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
#         yticks=species_order,
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, f'stacked_barplot__col-celtype__y-species__hue-kmer_category__{celltype_col}.pdf')
fig.savefig(pdf)

In [ ]:
figure_folder

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=5, figsize=(4, 6), sharex=True, sharey=True)


for (celltype, df), ax in zip(
    compartment_kmers_n_per_category_df_no_lemur .groupby(celltype_col), axes.flat
):
    
#     legend = ax.is_last_col() and ax.is_last_row()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y="species",
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=False,
        ax=ax,
    )
    ax.set(
        ylabel="species",
        xlabel='Percentage',
        title=celltype.replace("and", "\nand"),
    )
    
# Fix the legend so it's not on top of the bars.
# legend = ax.get_legend()
# legend.set_bbox_to_anchor((1, 1))
sns.despine()
fig.tight_layout()
pdf = os.path.join(figure_folder, f'stacked_barplot__col-celtype__y-species__hue-kmer_category__two_column__{celltype_col}.pdf')
fig.savefig(pdf)

In [ ]:
pdf

In [ ]:
f

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(12, 4), sharex=True, sharey=True)


for (species, df), ax in zip(
    diagnostic_kmers_n_per_category_df_no_lemur.groupby("species"), axes.flat
):
    legend = ax.is_last_col()
    # One liner to create a stacked bar chart.
    sns.histplot(
        df,
        y=celltype_col,
        hue="kmer_category",
        weights="percent_kmers",
        multiple="stack",
        palette="mako",
        linewidth=1,
        edgecolor="white",
        legend=legend,
        ax=ax,
    )
    ax.set(
        ylabel="percentage",
        title=species,
#         yticks=["bat", "human"],
    )
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

sns.despine()
fig.tight_layout()

pdf = os.path.join(figure_folder, 'stacked_barplot__col-species__y-celltype__hue-kmer_category.pdf')
fig.savefig(pdf)

# Subset to only diagnostic k-mers

In [ ]:
mouse_celltype_kmers['hashval'] = mouse_celltype_kmers.hashval.astype(str)

In [ ]:
describe(mouse_celltype_kmers)

In [ ]:
mouse_celltype_kmers.dtypes

In [ ]:
diagnostic_hashvals = set(mouse_celltype_kmers.hashval)
len(diagnostic_hashvals)

In [ ]:
hash2kmer_with_predictions.dtypes

In [ ]:
%%time

hash2kmer_with_predictions_diagnostic_kmers = hash2kmer_with_predictions.query('hashval in @diagnostic_hashvals')
describe(hash2kmer_with_predictions_diagnostic_kmers)

In [ ]:
diagnostic_kmers_n_per_category_df_no_lemur